In [1]:
import pandas as pd
import db_access

In [2]:
def get_raw_data(limit=None):
    engine = db_access.get_engine()
    query = "select * from usd_jpy"
    if (limit is not None and type(limit) is int):
        query = query + "limit " + str(limit)
    return pd.read_sql("SELECT * FROM usd_jpy", engine)

In [19]:
df = get_raw_data()
df = df[["datetime","open","high","low","close","volume"]].copy()
df.columns = ['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume']
df['Datetime'] = pd.to_datetime(df['Datetime'])
df = df.reset_index().set_index('Datetime')
df.head()

,index,Open,High,Low,Close,Volume
Datetime,,,,,,
2020-01-30 04:30:00,0,108.893,108.914,108.888,108.912,92
2020-01-30 04:35:00,1,108.914,108.923,108.909,108.921,121
2020-01-30 04:40:00,2,108.920,108.932,108.917,108.927,131
2020-01-30 04:45:00,3,108.926,108.928,108.904,108.908,143
2020-01-30 04:50:00,4,108.907,108.908,108.896,108.900,99


In [21]:
import talib as ta

In [22]:
def get_bband(close, timeperiod=20, nbdevup=3, nbdevdn=3, matype=0):
    gain = pd.DataFrame(close)
    gain.columns = ['close']
    upper, middle, lower = ta.BBANDS(gain.close, timeperiod, nbdevup , nbdevdn, matype)
    gain['bb_upper'] = upper
    gain['bb_lower'] = lower
    return gain['bb_upper'], gain['bb_lower']

In [36]:
def get_macd(close, fastperiod=6, slowperiod=13, signalperiod=4):
    macd, macdsignal, macdhist = ta.MACD(close, fastperiod=fastperiod, slowperiod=slowperiod, signalperiod=signalperiod)
    return macd, macdsignal

In [25]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

In [44]:
# https://bollinger-band.org/entry25.html
class MacdBbandStrategy(Strategy):
    bb_sigma2 = 2
    bb_sigma3 = 3
    bb_window = 9
    bb_matype = 0
    macd_fastperiod = 6
    macd_slowperiod = 13
    macd_signalperiod = 4
    
    profit = 30 * 0.01 # 利確30pips
    loss = 15 * 0.01 # 損切り15pips
    
    def init(self):
        self.bb2_upper, self.bb2_lower = self.I(get_bband, self.data.Close, self.bb_window, self.bb_sigma2, self.bb_sigma2, self.bb_matype)
        self.bb3_upper, self.bb3_lower = self.I(get_bband, self.data.Close, self.bb_window, self.bb_sigma3, self.bb_sigma3, self.bb_matype)
        self.macd, self.signal = self.I(get_macd, self.data.Close, self.macd_fastperiod, self.macd_slowperiod, self.macd_signalperiod)
        
        # 0: 上昇トレンド、 1: 下降トレンド、 99: トレンドなし
        self.trand = 99
        self.trand_change = 0

    def next(self):
        # macdのトレンド確認
        if (crossover(self.signal, self.macd)):
            self.trand = 0 if (self.signal > self.macd) else 1
        else:
            self.trand_change += 1
        
        # 5本をすぎると騙しが増えるのでトレンドとは考えない
        if (self.trand_change < 5):
            if (crossover(self.data.Close, self.bb2_upper)):
                self.buy(price=self.data.Close[-1], sl=self.data.Close[-1] - self.loss, tp=self.data.Close[-1] + self.profit)
            elif crossover(self.data.Close, self.bb2_lower):
                self.sell(price=self.data.Close[-1], sl=self.data.Close[-1] + self.loss, tp=self.data.Close[-1] - self.profit)
        else:
            self.trand_change = 0
            self.trand = 99
            

In [45]:
bt = Backtest(df, MacdBbandStrategy, cash=100000, commission=.00004)
output = bt.run()
print(output)

NameError: name 'loss' is not defined